<a href="https://colab.research.google.com/github/bettyteng21/1111-IR-hw3/blob/main/IR_hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyserini

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json
import numpy as np
!pip install faiss-cpu --no-cache
import faiss
from google.colab import drive
drive.mount('/content/drive/')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
f_origin = open('/content/drive/MyDrive/IR_data/pubmed 220 train.txt', 'r')
# f_origin = open('/content/drive/MyDrive/IR_data/short_ver.txt', 'r')
f_json = open('/content/IR_local_data/processed_data.json', 'w')

full_data = []

with f_origin as f:
  content = f.readlines()
  for i in str(content).split("###"):
    content = i.replace("\\n", "")\
          .replace("BACKGROUND\\t", "")\
          .replace("OBJECTIVE\\t", "")\
          .replace("METHODS\\t", "")\
          .replace("RESULTS\\t", "")\
          .replace("CONCLUSIONS\\t", "")
    # print(content.split())
    new_content = " ".join(content.split())
    id_doc = new_content.replace("',", "").replace("'", "")
    real_id = id_doc.split()[0]
    real_content = ' '.join(id_doc.split()[1:]).replace("]", "").replace('\"', "")
    # print(real_content)
    
    data = dict.fromkeys(["id", "contents"])
    data.update({'id': real_id, 'contents': real_content})
    
    # print(data)

    if(real_id !='['):
      full_data.append(data)

with f_json as jsonfile:
  json.dump(full_data, jsonfile)

f_origin.close()
f_json.close()


In [212]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input /content/IR_local_data \
  --index /content/indexes/sample_collection_json3 \
  --generator DefaultLuceneDocumentGenerator \
  --threads 1 \
  --storePositions --storeContents

2022-10-13 17:59:22,401 INFO  [main] index.IndexCollection (IndexCollection.java:248) - Setting log level to INFO
2022-10-13 17:59:22,404 INFO  [main] index.IndexCollection (IndexCollection.java:251) - Starting indexer...
2022-10-13 17:59:22,404 INFO  [main] index.IndexCollection (IndexCollection.java:252) - ============ Loading Parameters ============
2022-10-13 17:59:22,405 INFO  [main] index.IndexCollection (IndexCollection.java:253) - DocumentCollection path: /content/IR_local_data
2022-10-13 17:59:22,405 INFO  [main] index.IndexCollection (IndexCollection.java:254) - CollectionClass: JsonCollection
2022-10-13 17:59:22,406 INFO  [main] index.IndexCollection (IndexCollection.java:255) - Generator: DefaultLuceneDocumentGenerator
2022-10-13 17:59:22,406 INFO  [main] index.IndexCollection (IndexCollection.java:256) - Threads: 1
2022-10-13 17:59:22,407 INFO  [main] index.IndexCollection (IndexCollection.java:257) - Language: en
2022-10-13 17:59:22,407 INFO  [main] index.IndexCollection 

In [218]:
from pyserini.search.lucene import LuceneSearcher

searcher = LuceneSearcher('/content/indexes/sample_collection_json3')
hits = searcher.search('Cortical')

for i in range(len(hits)):
    print(f'{i+1:2} {hits[i].docid:4} {hits[i].score:.5f}')
    print("{")
    print('\t"id" : "' + hits[i].docid + '",')
    print('\t"contents" : "' + hits[i].contents + '",')
    print("}")


 1 23445918 5.14240
{
	"id" : "23445918",
	"contents" : "The purpose of this study was to determine variability in cortical volumetric bone density ( vBMD ) from a single slice tibia image over a range of vBMD readings and ages . Males and females ( N = 80 ; aged 6-80 ) were randomly selected from a previous study . Cortical vBMD at the anterior , posterior , medial , and lateral regions as well as the endocortical , mid-cortical , and pericortical regions of the cortical shell were determined using pQCT . Mean anterior ROI cortical vBMD ( 111111 mg/cm ( 3 ) ) was lower than the posterior and lateral ROIs ( 11697 mg/cm ( 3 ) and 11519 mg/cm ( 3 ) , respectively ) , ( p < 0.05 ) . Individuals with lower overall cortical vBMD and younger individuals had greater vBMD variability within the shell ( r = 0.37 , p = 0.004 ) . Difference between anterior and posterior regions was inversely associated with cortical vBMD and jump power ( r = 0.46 , p = 0.001 ) . Endocortical vBMD ( 10806 mg/cm )